## 임베딩 실습

### 활용 데이터: sample.csv
### 실습 개요
이 실습에서는 sample.csv 파일에 포함된 두 개의 텍스트 칼럼에 대해 Sentence-BERT를 활용하여 임베딩 값을 추출하고, 추출된 두 임베딩 간의 코사인 유사도를 계산하는 과제를 수행합니다. Sentence-BERT 라이브러리를 사용하여 텍스트 데이터를 고차원 벡터로 변환하고, 이 벡터들 사이의 유사도를 측정하는 방법을 실습합니다. 이때, Sentence-BERT는 huggingface에 있는 모델(bongsoo/kpf-sbert-v1.1 )을 활용해서 결과를 비교해 봅니다.

### 실습 절차
1. 라이브러리 설치 및 임포트: 필요한 라이브러리 (pandas, sentence_transformers)를 설치하고 임포트합니다.
2. CSV 파일 로드: pandas를 사용하여 주어진 CSV 파일을 읽고 DataFrame으로 변환합니다.
3. Sentence-BERT 모델 로드: sentence_transformers의 SentenceTransformer 클래스를 사용하여 사전 훈련된 모델을 로드합니다.
4. 텍스트 데이터 임베딩: DataFrame 내의 두 텍스트 칼럼에 대해 각각 임베딩을 계산합니다.
5. 코사인 유사도 계산: 추출된 두 임베딩 벡터 간의 코사인 유사도를 계산합니다.
6. 결과 확인 및 저장: 계산된 코사인 유사도를 새로운 열에 저장하고, 최종 결과를 cosine_similarity_results.csv 파일로 저장합니다.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install sentence-transformers

In [3]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import torch
from scipy.spatial.distance import cosine

In [4]:
df = pd.read_csv('/content/drive/MyDrive/NLP 실습/1 15/sample.csv')

In [5]:
model = SentenceTransformer('bongsoo/kpf-sbert-v1.1')

In [6]:
df.head(2)

,sentence1,sentence2
0,"억제 돌연변이 중 NAFM733은 0.1mg/ml로 γPGA를 생성했는데, 이는 다...","응답자의 일상 활동 뿐만 아니라 사회경제적, 개체군통계학적, 직업적 특성에 초점을 ..."
1,"총 알칼리도, 총 암모늄, tVFA와 pH의 농도는 모넨신 사료를 투여한 젖소의 거...","슬라이드를 1,200rpm에서 5분 동안 원심분리하여 건조하고 준비가 될 때까지 어..."


In [7]:
# 각 칼럼의 문장들에 대해 임베딩 계산
df['sentence1_embedding'] = df['sentence1'].apply(lambda x: model.encode(x))
df['sentence2_embedding'] = df['sentence2'].apply(lambda x: model.encode(x))

In [8]:
# 코사인 유사도 계산
def calculate_cosine_similarity(row):
    return 1 - cosine(row['sentence1_embedding'], row['sentence2_embedding'])

In [9]:
df['cosine_similarity'] = df.apply(calculate_cosine_similarity, axis=1)

In [10]:
df.to_csv('cosine_similarity_results.csv', index=False)

In [11]:
df.head(3)

,sentence1,sentence2,sentence1_embedding,sentence2_embedding,cosine_similarity
0,"억제 돌연변이 중 NAFM733은 0.1mg/ml로 γPGA를 생성했는데, 이는 다...","응답자의 일상 활동 뿐만 아니라 사회경제적, 개체군통계학적, 직업적 특성에 초점을 ...","[0.046548024, -0.424528, -2.9370493e-09, 0.268...","[-0.21460457, -0.012198336, 3.380338e-07, -0.0...",0.059022
1,"총 알칼리도, 총 암모늄, tVFA와 pH의 농도는 모넨신 사료를 투여한 젖소의 거...","슬라이드를 1,200rpm에서 5분 동안 원심분리하여 건조하고 준비가 될 때까지 어...","[-0.0672998, -0.25683028, -4.0544926e-08, 0.00...","[0.09949845, -0.58030057, 1.8800657e-07, 0.138...",0.330823
2,미생물학적 위험은 제조되는 치즈의 구체적인 특성에 따라 다르다.,축적된 증거는 장내 미생물 군집이 간암 발병에 중요한 역할을 한다는 것을 나타낸다.,"[-0.19007894, -0.7095868, 6.4557696e-08, 0.051...","[-0.025414037, -0.3333105, 1.3987189e-07, -0.0...",0.442524


## BLEU 및 SacreBLEU 점수 계산 실습


### 활용 데이터: sample.csv

### 실습 개요
이 실습에서는 주어진 csv 파일에 포함된 두 칼럼의 텍스트 데이터를 사용하여 BLEU(Bilingual Evaluation Understudy) 점수와 SacreBLEU 점수를 계산하는 과제를 수행합니다. 이때, BLUE 스코어 계산시 klue/roberta-base 토크나이저를 활용해 계산합니다.

### 실습 절차
1. 라이브러리 설치 및 임포트: 필요한 라이브러리 (pandas, nltk, sacrebleu)를 설치하고 임포트합니다.
2. CSV 파일 로드: pandas를 사용하여 주어진 CSV 파일을 읽고 DataFrame으로 변환합니다.
3. BLEU 점수 계산: nltk 라이브러리의 sentence_bleu 함수를 사용하여 BLEU 점수를 계산합니다.
4. SacreBLEU 점수 계산: sacrebleu 라이브러리를 사용하여 SacreBLEU 점수를 계산합니다.
5. 결과 저장 및 비교: 두 방법으로 계산된 BLEU 점수를 각각 새로운 열에 저장하고, 결과를 비교 분석합니다.
6. 최종 결과 저장: 계산된 BLEU 및 SacreBLEU 점수가 포함된 DataFrame을 bleu_sacrebleu_results.csv 파일로 저장합니다.

In [12]:
!pip install konlpy sacrebleu nltk

In [13]:
from nltk.translate.bleu_score import sentence_bleu
from konlpy.tag import Okt
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import sacrebleu

In [14]:
df = pd.read_csv('/content/drive/MyDrive/NLP 실습/1 15/sample.csv')

In [15]:
tokenizer = AutoTokenizer.from_pretrained("klue/roberta-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [16]:
# 각 칼럼의 문장들에 대해 토큰화
df['sentence1_tokenize'] = df['sentence1'].apply(lambda x: tokenizer.tokenize(x))
df['sentence2_tokenize'] = df['sentence2'].apply(lambda x: tokenizer.tokenize(x))

In [17]:
df.head(2)

,sentence1,sentence2,sentence1_tokenize,sentence2_tokenize
0,"억제 돌연변이 중 NAFM733은 0.1mg/ml로 γPGA를 생성했는데, 이는 다...","응답자의 일상 활동 뿐만 아니라 사회경제적, 개체군통계학적, 직업적 특성에 초점을 ...","[억제, 돌연변이, 중, NA, ##F, ##M, ##73, ##3, ##은, 0,...","[응답자, ##의, 일상, 활동, 뿐, ##만, 아니, ##라, 사회, ##경제, ..."
1,"총 알칼리도, 총 암모늄, tVFA와 pH의 농도는 모넨신 사료를 투여한 젖소의 거...","슬라이드를 1,200rpm에서 5분 동안 원심분리하여 건조하고 준비가 될 때까지 어...","[총, 알칼리, ##도, ,, 총, 암, ##모, ##늄, ,, t, ##V, ##...","[슬라이드, ##를, 1, ,, 200, ##r, ##p, ##m, ##에서, 5,..."


In [18]:
# BLEU 점수 계산
df['sentence_bleu_score'] = df.apply(lambda x: sentence_bleu([x['sentence1_tokenize']], x['sentence2_tokenize']), axis=1)

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

In [19]:
df.head(2)

,sentence1,sentence2,sentence1_tokenize,sentence2_tokenize,sentence_bleu_score
0,"억제 돌연변이 중 NAFM733은 0.1mg/ml로 γPGA를 생성했는데, 이는 다...","응답자의 일상 활동 뿐만 아니라 사회경제적, 개체군통계학적, 직업적 특성에 초점을 ...","[억제, 돌연변이, 중, NA, ##F, ##M, ##73, ##3, ##은, 0,...","[응답자, ##의, 일상, 활동, 뿐, ##만, 아니, ##라, 사회, ##경제, ...",3.443744e-155
1,"총 알칼리도, 총 암모늄, tVFA와 pH의 농도는 모넨신 사료를 투여한 젖소의 거...","슬라이드를 1,200rpm에서 5분 동안 원심분리하여 건조하고 준비가 될 때까지 어...","[총, 알칼리, ##도, ,, 총, 암, ##모, ##늄, ,, t, ##V, ##...","[슬라이드, ##를, 1, ,, 200, ##r, ##p, ##m, ##에서, 5,...",1.663000e-78


In [20]:
# 각 칼럼의 문장들에 대해 토큰화
df['sentence1_concat'] = df['sentence1_tokenize'].apply(lambda x: [' '.join(x)])
df['sentence2_concat'] = df['sentence2_tokenize'].apply(lambda x: [' '.join(x)])

In [21]:
df['sacrebleu'] = df.apply(lambda x: sacrebleu.corpus_bleu(x['sentence1_concat'], [x['sentence2_concat']]).score, axis=1)

In [22]:
df.head(2)

,sentence1,sentence2,sentence1_tokenize,sentence2_tokenize,sentence_bleu_score,sentence1_concat,sentence2_concat,sacrebleu
0,"억제 돌연변이 중 NAFM733은 0.1mg/ml로 γPGA를 생성했는데, 이는 다...","응답자의 일상 활동 뿐만 아니라 사회경제적, 개체군통계학적, 직업적 특성에 초점을 ...","[억제, 돌연변이, 중, NA, ##F, ##M, ##73, ##3, ##은, 0,...","[응답자, ##의, 일상, 활동, 뿐, ##만, 아니, ##라, 사회, ##경제, ...",3.443744e-155,[억제 돌연변이 중 NA ##F ##M ##73 ##3 ##은 0 . 1 ##m #...,"[응답자 ##의 일상 활동 뿐 ##만 아니 ##라 사회 ##경제 ##적 , 개체 #...",9.549902
1,"총 알칼리도, 총 암모늄, tVFA와 pH의 농도는 모넨신 사료를 투여한 젖소의 거...","슬라이드를 1,200rpm에서 5분 동안 원심분리하여 건조하고 준비가 될 때까지 어...","[총, 알칼리, ##도, ,, 총, 암, ##모, ##늄, ,, t, ##V, ##...","[슬라이드, ##를, 1, ,, 200, ##r, ##p, ##m, ##에서, 5,...",1.663000e-78,"[총 알칼리 ##도 , 총 암 ##모 ##늄 , t ##V ##FA ##와 p ##...","[슬라이드 ##를 1 , 200 ##r ##p ##m ##에서 5 ##분 동안 원심...",19.875021


In [23]:
df.to_csv('bleu_sacrebleu_results.csv', index=False)